In [1]:
import requests
import time
from time import sleep
from bs4 import BeautifulSoup

In [2]:
#找出魔鏡歌詞網中有列出的全部理想混蛋的歌的網址
link = []

for i in range(1, 21):
    if i == 9:
        link.append('https://mojim.com/twy219017x9x1.htm')
        link.append('https://mojim.com/twy219017x9x2.htm')
    elif i == 11:
        for j in range(1, 12):
            link.append(f'https://mojim.com/twy219017x{i}x{j}.htm')
    elif i == 18:
        for j in range(1, 11):
            link.append(f'https://mojim.com/twy219017x{i}x{j}.htm')
    elif i != 2:
        link.append(f'https://mojim.com/twy219017x{i}x1.htm')

In [3]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'
}
bestards = []
for url in link:
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        #print(f'{url} 請求成功！') #註:因為確認都顯示成功，所以就不print了
        soup = BeautifulSoup(response.text, 'html.parser')
        bestards.append(soup)
    #else:
        #print(f'{url} 請求失敗，狀態碼為 {response.status_code}。')

In [4]:
for url in link:  
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    tag_div = soup.find_all('div', {'id': 'fsZ', 'class': 'fsZ'})
    for div in tag_div:
        tag_dl = div.find_all('dl', {'id': 'fsZx1', 'class': 'fsZx1'})
        #print(tag_dl)

In [5]:
#存成csv檔
import csv
data = []
for url in link:
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    tag_div = soup.find_all('div', {'id': 'fsZ', 'class': 'fsZ'})
    for div in tag_div:
        tag_dl = div.find_all('dl', {'id': 'fsZx1', 'class': 'fsZx1'})
        for dl in tag_dl:
            # extract the data from each <dl> tag
            # and append it to the data list
            data.append([dt.text.strip() for dt in dl.find_all('dt')])
            data.append([dd.text.strip() for dd in dl.find_all('dd')])

# write the data to a CSV file
with open('data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(data)

#### 爬蟲完成，接下來進行斷詞

In [7]:
import pandas as pd

bestard = pd.read_csv('data.csv')
bestard.head()

,行星
0,作詞：邱建豪作曲：邱建豪、郭哲成編曲：理想混蛋主唱：邱建豪沒有行李 沒有目的一個人旅行在無邊...
1,不是因為天氣晴朗才愛你
2,演唱：邱建豪作詞：邱建豪作曲：邱建豪編曲：理想混蛋不是因為天氣晴朗才愛你不是因為看見星星才想...
3,獨角仙的夢 (Kabutomushi's Dream)
4,作詞：邱建豪作曲：邱建豪編曲：理想混蛋驕傲的活著 向晚的天空裡飛身體還殘留著 土壤氣味高樓天...


In [8]:
import jieba
import jieba.analyse
import numpy
from collections import Counter

In [13]:
bestard['seg'] = ""
bestard.head()

,行星,seg
0,作詞：邱建豪作曲：邱建豪、郭哲成編曲：理想混蛋主唱：邱建豪沒有行李 沒有目的一個人旅行在無邊...,
1,不是因為天氣晴朗才愛你,
2,演唱：邱建豪作詞：邱建豪作曲：邱建豪編曲：理想混蛋不是因為天氣晴朗才愛你不是因為看見星星才想...,
3,獨角仙的夢 (Kabutomushi's Dream),
4,作詞：邱建豪作曲：邱建豪編曲：理想混蛋驕傲的活著 向晚的天空裡飛身體還殘留著 土壤氣味高樓天...,


In [14]:
index = 0
for documents in bestard['行星'].astype(str):
    #seg_list = jieba.analyse.extract_tags(documents, 150)
    seg_list = jieba.lcut(documents)
    bestard['seg'].iloc[index] = seg_list
    index += 1

In [15]:
bestard['seg']

0     [作詞, ：, 邱建豪, 作曲, ：, 邱建豪, 、, 郭哲成, 編曲, ：, 理想, 混蛋...
1                             [不是, 因為, 天氣, 晴朗, 才, 愛, 你]
2     [演唱, ：, 邱建豪, 作詞, ：, 邱建豪, 作曲, ：, 邱建豪, 編曲, ：, 理想...
3     [獨角, 仙, 的, 夢,  , (, Kabutomushi, ', s,  , Drea...
4     [作詞, ：, 邱建豪, 作曲, ：, 邱建豪, 編曲, ：, 理想, 混蛋, 驕傲, 的,...
                            ...                        
72    [作詞, ：, 邱建豪, 、, 郭哲成, 作曲, ：, 邱建豪, 、, 郭哲成, 冰河期, ...
73                                            [屬, 於, 你]
74    [電影,  , 一周, 的, 朋友,  , 插曲, 作詞, ：, 林孝謙, 作曲, ：, 盧...
75                                     [Easy,  , Peasy]
76    [作詞, ：, 邱建豪, 、, 盧可沛, 、, 李建廷, 、, 郭哲成, 作曲, ：, 盧可...
Name: seg, Length: 77, dtype: object

In [12]:
bestard.to_csv("lyrics.csv", index=False)

         1010
的         535
你         373
我         299
[         175
:         175
]         175
在         148
了         121
，         114
：         106
是          89
更          84
不          80
也          74
邱建豪        71
有          70
再          69
說          62
就          59
都          54
我們         53
什麼         52
著          51
卻          49
多          48
好          46
.          43
作詞         42
作曲         42
歌詞         42
歌          41
02         40
。          39
Mojim      39
com        39
詞網         39
　          39
※          39
魔鏡         39
誰          39
啦          39
沒          39
03         38
、          38
詳盡         37
01         37
00         37
怎麼         36
人          36
dtype: int64